In [ ]:
pip install tweepy

In [ ]:
pip install kafka-python

In [ ]:
pip install scylla-driver

In [ ]:
import json
from datetime import datetime
from kafka import KafkaProducer, KafkaConsumer
import tweepy
import os
import pandas as pd
import sys
import hashlib

In [ ]:
class Producers:
    def __init__(self):
        self.broker = '172.24.160.5:9092'
        self.topico = 'twitter_news'
        self.resp = ""
        self.prod = KafkaProducer(bootstrap_servers=[self.broker],
                                  value_serializer=lambda v: json.dumps(v).encode('utf-8'))

    def send_msg(self):
        data_hora = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        dados = {"tweet": self.resp, "horario": data_hora}

        self.prod.send(self.topico, value=dados)

In [ ]:
def create_user_locs(tweets):
    user_locs = []
    contadorTweets = 0
    prod = Producers()
    for tweet in tweets:
        username = str(tweet.user.screen_name.lower().encode('ascii',errors='ignore'))
        description = str(tweet.user.description.lower().encode('ascii',errors='ignore'))
        location = str(tweet.user.location.lower().encode('ascii',errors='ignore'))
        following = str(str(tweet.user.friends_count).lower().encode('ascii',errors='ignore'))
        followers = str(str(tweet.user.followers_count).lower().encode('ascii',errors='ignore'))
        totaltweets = str(str(tweet.user.statuses_count).lower().encode('ascii',errors='ignore'))
        retweetcount = str(str(tweet.retweet_count).lower().encode('ascii',errors='ignore'))
        text = str(tweet.text.lower().encode('ascii',errors='ignore'))
        th_tweet = [username, description, location, following,followers, totaltweets, retweetcount, text]
        user_locs.append(th_tweet)
        #grava dados no kafka simulando um producer
        prod.resp = tweet._json
        prod.send_msg()
        contadorTweets += 1

    return user_locs, contadorTweets

In [ ]:
def hash_username(user_list):
    length = range(len(user_list))
    for i in length:
        user_list[i][0] = hashlib.md5(user_list[i][0].encode()).hexdigest()
    return user_list

In [ ]:
def create_csv(data, file_name):
    dirarq = '/home/docker_worker/work/'
    df = pd.DataFrame(data=data,columns=['username', 'description', 'location', 'following',
                                'followers', 'totaltweets', 'retweetcount', 'text'])

    df.to_csv(dirarq + file_name)

In [15]:
if __name__ == "__main__":
    consumer_key = "----"
    consumer_secret = "----"
    access_token = "333333"
    access_token_secret = "----"

    hashtag = "#Covid" + " -filter:retweets"
    numbertweets = 100
    file_name = "twitter_arq.txt"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)

    tweets = tweepy.Cursor(api.search_tweets,q=hashtag,lang="pt").items(numbertweets)

    data, qtdTweet = create_user_locs(tweets)
    data_hash = hash_username(data)
    create_csv(data_hash, file_name)

    print("Foram coletados " + str(qtdTweet) + " tweets.")

Foram coletados 100 tweets.
